<a href="https://colab.research.google.com/github/Pt-home/Notebooks/blob/main/Nomades2_0_Rashid_06(rotation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import numpy
# библиотека scipy.special содержит сигмоиду expit()
import scipy.special
import matplotlib.pyplot
%matplotlib inline

In [21]:
# определение класса нейронной сети class neuralNetwork:
class neuralNetwork:

  def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate) :
    # задать количество узлов во входном, скрытом и выходном слое
    self.inodes = inputnodes
    self.hnodes = hiddennodes
    self.onodes = outputnodes

    # коэффициент обучения
    self.lr = learningrate

    #матрицы весов
    self.wih = numpy.random.rand(self.hnodes, self.inodes) - 0.5
    self.who = numpy.random.rand(self.onodes, self.hnodes) - 0.5
    #self.wih = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
    #self.who = numpy.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))

    # использование сигмоиды в качестве функции активации
    self.activation_function = lambda x: scipy.special.expit(x)

  def train(self, inputs_list, targets_list):

    # Прямой проход

    # преобразовать список входных значений в двухмерный массив
    inputs =  numpy.array(inputs_list, ndmin = 2).T
    targets = numpy.array(targets_list, ndmin = 2).T

    # рассчитать входящие сигналы для скрытого слоя
    hidden_inputs = numpy.dot(self.wih, inputs)
    # рассчитать исходящие сигналы для скрытого слоя
    hidden_outputs = self.activation_function(hidden_inputs)

    # рассчитать входящие сигналы для выходного слоя
    final_inputs = numpy.dot(self.who, hidden_outputs)
    # рассчитать исходящие сигналы для выходного слоя
    final_outputs = self.activation_function(final_inputs)

    # Анализ и коррекция весов

    # ошибка = целевое значение - фактическое значение
    output_errors = targets - final_outputs

    # ошибки скрытого слоя - это ошибки output_errors, распределенные пропорционально весовым коэффициентам связей и рекомбинированные на скрытых узлах
    hidden_errors = numpy.dot(self.who.T, output_errors)

    # обновить весовые коэффициенты связей между скрытым и выходным слоями
    self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
    # обновить весовые коэффициенты связей между входным и скрытым слоями
    self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))

  def query(self, inputs_list):
    # преобразовать список входных значений в двухмерный массив
    inputs = numpy.array(inputs_list, ndmin = 2).T

    # рассчитать входящие сигналы для скрытого слоя
    hidden_inputs = numpy.dot(self.wih, inputs)
    # рассчитать исходящие сигналы для скрытого слоя
    hidden_outputs = self.activation_function(hidden_inputs)

    # рассчитать входящие сигналы для выходного слоя
    final_inputs = numpy.dot(self.who, hidden_outputs)
    # рассчитать исходящие сигналы для выходного слоя
    final_outputs = self.activation_function(final_inputs)

    return final_outputs



In [22]:
# количество входных, скрытых и выходных узлов
input_nodes = 784
hidden_nodes = 200
output_nodes = 10
# коэффициент обучения равен 0,01
learning_rate = 0.01

epochs = 10

In [23]:
# создать экземпляр нейронной сети
n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [24]:
# загрузить в список тренировочный набор данных CSV-файла набора MNIST
training_data_file = open("mnist_train.csv", "r")
training_data_list = training_data_file.readlines()
training_data_file.close()

In [25]:
# загрузить в список тестовый набор данных CSV-файла набора MNIST
test_data_file = open("mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [26]:
# тренировка нейронной сети
import time

start = time.time()
print("Начало тренировки на", epochs, "эпох в", time.ctime(start))

for e in range(epochs):

    # перебрать все записи в тренировочном наборе данных
    for record in training_data_list:

      # получить список значений, используя символы запятой (1,1) в качестве разделителей
      all_values = record.split(",")

      # масштабировать и сместить входные значения
      inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01

      # создать целевые выходные значения (все равны 0,01, за исключением желаемого маркерного значения, равного 0,99)
      targets = numpy.zeros(output_nodes) + 0.01
      index = int(all_values[0])
      targets[index] = 0.99

      n.train(inputs, targets)

      angle = 10
      ## create rotated variations rotated anticlockwise by x degrees
      inputs_plusx_img = scipy.ndimage.rotate(inputs.reshape(28,28), angle,
                                                            cval=0.01, order=1, reshape=False)
      n.train(inputs_plusx_img.reshape(784), targets)
      # rotated clockwise by x degrees
      inputs_minusx_img = scipy.ndimage.rotate(inputs.reshape(28,28), -angle,
                                                             cval=0.01, order=1, reshape=False)
      n.train(inputs_minusx_img.reshape(784), targets)

    scorecard = []

    for record in test_data_list:
         all_values = record.split(",")
         inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01

         outputs = n.query(inputs)
         label = numpy.argmax(outputs)
         correct_label = int(all_values[0])

         if correct_label==label:
            scorecard.append(1)
         else:
            scorecard.append(0)

    finish = time.time()
    print(f"Эпоха № {e+1}. Эффективность: {sum(scorecard) / len(scorecard)}.  В {time.ctime(finish)}")

    wih = n.wih.flatten()
    wih = list(wih)
    who = n.who.flatten()
    who = list(who)

    f = open('wih_' + str(hidden_nodes) + '_' + str(learning_rate) + '_' + str(e+1) + '.dat','w')
    for i in range(len(wih)-1):
        f.write(str(wih[i])+',')
    f.write(str(wih[-1]))
    f.close()

    f = open('who_' + str(hidden_nodes) + '_' + str(learning_rate) + '_' + str(e+1) + '.dat','w')
    for i in range(len(who)-1):
        f.write(str(who[i])+',')
    f.write(str(who[-1]))
    f.close()

Начало тренировки на 10 эпох в Sun Sep 10 09:38:55 2023
Эпоха № 1. Эффективность: 0.9433.  В Sun Sep 10 09:42:20 2023
Эпоха № 2. Эффективность: 0.9577.  В Sun Sep 10 09:45:44 2023
Эпоха № 3. Эффективность: 0.9646.  В Sun Sep 10 09:49:09 2023
Эпоха № 4. Эффективность: 0.9687.  В Sun Sep 10 09:52:34 2023
Эпоха № 5. Эффективность: 0.9717.  В Sun Sep 10 09:56:01 2023
Эпоха № 6. Эффективность: 0.9742.  В Sun Sep 10 09:59:27 2023
Эпоха № 7. Эффективность: 0.9756.  В Sun Sep 10 10:02:53 2023
Эпоха № 8. Эффективность: 0.9769.  В Sun Sep 10 10:06:20 2023
Эпоха № 9. Эффективность: 0.977.  В Sun Sep 10 10:09:46 2023
Эпоха № 10. Эффективность: 0.9771.  В Sun Sep 10 10:13:11 2023
